In [ ]:
from __future__ import absolute_import, print_function, division
from pymba import *
import numpy as np
import cv2
import time

In [6]:
# start Vimba
with Vimba() as vimba:
    # get system object
    system = vimba.getSystem()

    # list available cameras (after enabling discovery for GigE cameras)
    if system.GeVTLIsPresent:
        system.runFeatureCommand("GeVDiscoveryAllOnce")
        time.sleep(0.2)
    cameraIds = vimba.getCameraIds()
    for cameraId in cameraIds:
        print('Camera ID:', cameraId)

    # get and open a camera
    camera0 = vimba.getCamera(cameraIds[0])
    camera0.openCamera()

#     # list camera features
#     cameraFeatureNames = camera0.getFeatureNames()
#     for name in cameraFeatureNames:
#         print('Camera feature:', name)

#     # read info of a camera feature
#     featureInfo = camera0.getFeatureInfo('AcquisitionMode')
#     for field in featureInfo.getFieldNames():
#        print(field, '--', getattr(featureInfo, field))

    # get the value of a feature


    # set the value of a feature
    num_frames = 10000
    camera0.AcquisitionMode = 'MultiFrame'
    camera0.AcquisitionFrameCount = num_frames
    
#     # get the value of a feature
#     print(camera0.AcquisitionFrameRate)
#     print(camera0.AcquisitionFrameCount)


    # create new frames for the camera
    frame0 = camera0.getFrame()    # creates a frame
    frame1 = camera0.getFrame()    # creates a second frame

    # announce frame
    frame0.announceFrame()

    # capture a camera image
    camera0.startCapture()
    camera0.runFeatureCommand('AcquisitionStart')
    frame0.queueFrameCapture()

    images = []
    count = 0
    
    start = time.time()

    while count < num_frames:
        frame0.waitFrameCapture()
        frame0.queueFrameCapture()
        
        # get image data...
        imgData = frame0.getBufferByteData()
        images.append(np.copy(imgData))
        count += 1
        if count % 1000 == 0:
            print(count)
    
    end = time.time()
    print("Time elapsed:", end - start)

    camera0.endCapture()
    camera0.runFeatureCommand('AcquisitionStop')
    # clean up after capture
    camera0.revokeAllFrames()

    # close camera
    camera0.closeCamera()

Camera ID: DEV_1AB2280005D8
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000


In [7]:
# save frames as images
for i in range(len(images)):
    imgData = np.ndarray(buffer = images[i],
                                       dtype = np.uint8,
                                       shape = (frame0.height,
                                                frame0.width,
                                                1))
    cv2.imwrite('foo{}.png'.format(i), imgData)